In [2]:
import pandas as pd
df = pd.read_csv("Products.csv", encoding='ISO-8859-1')
df.dtypes

ProductKey         int64
Product Name      object
Brand             object
Color             object
Unit Cost USD     object
Unit Price USD    object
SubcategoryKey     int64
Subcategory       object
CategoryKey        int64
Category          object
dtype: object

In [3]:

# Remove dollar sign, commas, and strip any trailing or leading spaces
df['Unit Cost USD'] = df['Unit Cost USD'].str.replace('$', '').str.replace(',', '').str.strip()
df['Unit Price USD'] = df['Unit Price USD'].str.replace('$', '').str.replace(',', '').str.strip()

# Convert the cleaned strings to floats
df['Unit Cost USD'] = df['Unit Cost USD'].astype(float)
df['Unit Price USD'] = df['Unit Price USD'].astype(float)


In [19]:
df

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [20]:
df.dtypes

ProductKey          int64
Product Name       object
Brand              object
Color              object
Unit Cost USD     float64
Unit Price USD    float64
SubcategoryKey      int64
Subcategory        object
CategoryKey         int64
Category           object
dtype: object

In [4]:
df.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [5]:
# Save the cleaned dataset to a new CSV file
df.to_csv('Cleaned_Products.csv', index=False)

In [6]:
df.dtypes

ProductKey          int64
Product Name       object
Brand              object
Color              object
Unit Cost USD     float64
Unit Price USD    float64
SubcategoryKey      int64
Subcategory        object
CategoryKey         int64
Category           object
dtype: object

In [7]:
import pandas as pd
import pymysql
# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='password', database="project")

# Load CSV data
df = pd.read_csv("Cleaned_Products.csv")
df = df.dropna()  # Drop null values

# Replace data types from pandas to MySQL
a = ",".join(f"`{i}` {j}" for i, j in zip(df.columns, df.dtypes)).replace("object", "TEXT").replace("int64", "INT").replace("float64","FLOAT")

# Use a valid table name
table_name = "Products"

# Create the table
create_table_query = f"CREATE TABLE `{table_name}` ({a})"
with myconnection.cursor() as cursor:
    cursor.execute(create_table_query)

# Insert data into the table
for i in range(len(df)):
    insert_query = f"INSERT INTO `{table_name}` VALUES {tuple(df.iloc[i])}"
    with myconnection.cursor() as cursor:
        cursor.execute(insert_query)
    myconnection.commit()

myconnection.close()